### **Przegląd Wbudowanych Sekwencji**

Biblioteka standardowa oferuje bogaty wybór sekwencji zaimplementowanych w języku C:
- Sekwencje kontenerowe - mogą przechowywać elementy różnych typów, włącznie z innymi kontenerami. Przykłady to `list`, `tuple` i `collections.deque`.
- Sekwencje płaskie - przechowują elementy jednego typu. Przykłady to `str`, `bytes` i `array.array`.

Sekwencje kontenerowe przechowują odwołania do zawartych w sobie obiektów, które mogą być dowolnego typu, natomiast sekwencje płaskie fizycznie przechowują wartości poszczególnych elementów we własnej przestrzeni pamięci, a nie jako oddzielne obiekty. Sekwencje płaskie są bardziej upakowane i ograniczone do przechowywania prymitywnych wartości, takich jak znaki, bajty i liczby.

Każdy obiekt Pythona zawera w pamięci nagłówek z metadanymi. Najprostszy obiekt Pythona czyli `float`, ma pole wartości oraz dwa pola metadanych:
- `ob_refcnt` - licznik referencji do obiektu,
- `ob_type` - wskaźnik do typu obiektu,
- `ob_fval` - liczba `double` języka C, przechowująca wartość.

Innym sposobem jest podział sekwencji na ich zmienność:
- Sekwencje zmienne - przykładowo `list`, `bytearray`, `array.array` i `collections.deque`.
- Sekwencje niezmienne - przykładowo `tuple`, `str`, `bytes`.

<p align="center">
  <img src="seq.png"/>
</p>

Sekwencje zmienne implementują wszystkie metody sekwencji niezmiennych oraz dodają nowe. Konkretne wbudowane typy sekwencji nie są faktycznie podklasami ABC `Sequence` i `MutableSequence` ale są podklasami wirtualnymi zarejestrowanymi względem tych klas ABC.

### **Wyrażenia Listowe**

Wyrażenia listowe mają tylko jedno zastosowanie: mają budować listy. Wynikowa lista porządkowana jest tak jakby pętle `for` były zagnieżdżone w tej samej kolejności, w jakiej występują w wyrażeniu listowym.

In [84]:
listcomp = [chr(i) + str(j) for i in range(97, 100) for j in range(1, 4)]
listcomp

['a1', 'a2', 'a3', 'b1', 'b2', 'b3', 'c1', 'c2', 'c3']

Aby wypełnić inne typy sekwencji, należy użyć wyrażenia generatora. Oczywiście można użyć wyrażenia listowego ale wyrażenie generatora oszczędza pamięć, ponieważ generuje pojedyncze elementy przy użyciu protokołu iteratora. **Nie budujemy listy tylko po to aby załadować ją do innego konstruktora!**

### **Wyrażenia Listowe - Zasięg Lokalny**

Wyrażenia listowe, wyrażenia generatora i ich kuzyni, mają zasięg lokalny do przechowywania zmiennych przypisanych w klauzuli `for`.

In [85]:
x = "ABC"
codes = [ord(x) for x in x]

In [86]:
x  # Zmienna x nie jest zmieniana

'ABC'

In [87]:
codes = [last := ord(c) for c in x]  # Przypisanie operatorem morsa (walrus operator)
codes

[65, 66, 67]

In [88]:
last  # Zmienna last jest dostępna poza listą składaną

67

### **Krotki**

Interpreter Pythona i biblioteka standardowa szeroko wykorzystują krotki jako niezmienne listy. Zapewnia to dwie kluczowe korzyści:
- Przejrzystość - gdy widzimy w kodzie krotkę, wiemy, że jej długość się nigdy nie zmieni.
- Wydajność - krotka zajmuje mniej pamięci niż lista o tej samej długości, co pozwala dokonać Pythonowi optymalizacji.

**Niezmienność krotki dotyczy tylko referencji, które zawiera.** Odnośników znajdujących się w krotce nie można usunąć ani zastąpić. Jeśli jednak któraś z tych referencji wskazuje modyfikowalny obiekt i ten obiekt zostanie zmieniony to wartość krotki również się zmieni.

Wydajność krotek:
- Dla danej krotki `x`, `tuple(x)` po prostu zwraca referencję do samego `x`. Nie ma potrzeby kopiowania. Dla kontrastu gdy mamy listę `x`, konstruktor `list(x)` utworzy nową kopię `x`.
- Dzięki ustalonej długości instancja krotki ma alokowane dokładnie tyle miejsca ile potrzebuje. Natomiast instancje list są alokowane z zapasowym miejscem.
- Referencje do elementów w krotce są przechowywane w tablicy w strukturze krotki, podczas gdy lista utrzymuje wskaźnik do tablicy referencji przechowywanej w innym miejscu. 

### **Dopasowywanie Wzorców**

Dopasowywanie wzorców jest dostępne w Pythonie od wersji 3.10. Na pierwszy rzut oka `match/case` może wyglądać podobnie do instrukcji `switch/case` z języków C/C++. Jednakże jednym z kluczowych ulepszeń `match` jest destrukturyzacja - bardziej zaawansowana forma rozpakowywania. 

In [89]:
def handle_command(message):
    # Wyrażenie po słowie kluczowym match to temat, dane które Python
    # próbuje dopasować do wzorców w każdej klauzuli case.
    match message:
        # Ten wzorzec pasuje do każdego tematu, który jest sekwencją 3 elementów.
        # Pierwszym elementem musi być łańcuch "BEEPER". Drugim i trzecim elementem
        # może być cokolwiek i zostanie to przypisane do zmiennych frequency i times
        # w takiej właśnie kolejności.
        case ["BEEPER", frequency, times]:
            print(f"Beeping {times} times at {frequency} Hz")
        # Ten wzorzec pasuje do każdego tematu, który jest sekwencją 2 elementów.
        # Pierwszym elementem musi być łańcuch "NECK". Drugim elementem może być
        # cokolwiek i zostanie to przypisane do zmiennej angle.
        case ["NECK", angle]:
            print(f"Moving neck to {angle} degrees")
        # To pasuje do każdego tematu, który jest sekwencją 3 elementów, gdzie
        # pierwszym elementem jest łańcuch "LED". Jeśli liczba elementów nie pasuje
        # to Python przejdzie do kolejnego wzorca.
        case ["LED", ident, intensity]:
            print(f"Setting LED {ident} to {intensity}")
        # Kolejny wzorzec dla "LED", ale tym razem oczekujemy 5 elementów.
        case ["LED", ident, red, green, blue]:
            print(f"Setting LED {ident} to RGB({red}, {green}, {blue})")
        # Domyślny przypadek, zostanie dopasowany do dowolnego tematu, który nie pasuje
        # do żadnego z powyższych wzorców.
        case _:
            print("Unknown command")

In [90]:
handle_command(["BEEPER", 440, 3])

Beeping 3 times at 440 Hz


In [91]:
handle_command(["NECK", 90])

Moving neck to 90 degrees


In [92]:
handle_command(["LED", 1, 255])

Setting LED 1 to 255


In [93]:
handle_command(["LED", 1, 195, 24, 254])

Setting LED 1 to RGB(195, 24, 254)


In [94]:
handle_command(["INVALID", "COMMAND"])

Unknown command


In [95]:
metro_areas = [
    ("Tokyo", "JP", 36.933, (35.689722, 139.691667)),
    ("Delhi NCR", "IN", 21.935, (28.613889, 77.208889)),
    ("Mexico City", "MX", 20.142, (19.433333, -99.133333)),
    ("New York-Newark", "US", 20.104, (40.808611, -74.020386)),
    ("Sao Paulo", "BR", 19.649, (-23.547778, -46.635833)),
]


def display(metro_areas):
    print(f"{'':15} | {'lat.':^9} | {'lon.':^9}")
    for record in metro_areas:
        match record:
            # Ten wzorzec pasuje do każdego tematu, który jest sekwencją 4 elementów,
            # przy czym ostatni element musi być sekwencją 2 elementów.
            # Ta klauzula ma dwie części: wzorzec i strażnik. Strażnik to warunek
            # logiczny, który musi być spełniony jeśli wzorzec pasuje.
            # Symbol _ ma specjalne znaczenie - dopasowuje dowolny pojedynczy
            # element na tej pozycji ale nigdy nie jest wiązany z wartością elementu.
            # Ponadto może pojawiać się we wzorcu więcej niż raz.
            case [name, _, _, (lat, lon)] if lon <= 0:
                print(f"{name:15} | {lat:9.4f} | {lon:9.4f}")

            # Można powiązać dowolną część wzorca ze zmienną za pomocą słowa kluczowego as.
            # case [name, _, _, (lat, lon) as coords] if lon <= 0:
            #     pass

            # Wzorce można uczynić bardziej szczegółowymi, dodając informacje o typie.
            # W kontekście wzorca, składnia ta wykonuje sprawdzenie typu w czasie wykonania.
            # case [str(name), _, _, (float(lat), float(lon))] if lon <= 0:
            #     pass


display(metro_areas)

                |   lat.    |   lon.   
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


Wzorzec sekwencji może pasować do wystąpień większości rzeczywistych lub wirtualnych podklas `collections.abc.Sequence` z wyjątkiem `str`, `bytes` i `bytearray`. Te ostatnie nie są obsługiwane w kontekście `match/case`. Temat `match` jest jednym z tych typów, które są traktowane jako wartość atomowa. Podobnie jak liczba 123 jest traktowana jako jedna wartość a nie sekwencja cyfr.

### **Budowanie Listy List**

In [96]:
board = [["_"] * 3 for _ in range(3)]
board[0][0] = "X"
board

[['X', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [97]:
# Poniższy kod jest tożsamy z powyższym.
board = []
for _ in range(3):
    row = ["_"] * 3
    board.append(row)

In [98]:
# To tworzy listę trzech referencji do tego samego obiektu listy.
board = [["_"] * 3] * 3
board[0][0] = "X"
board

[['X', '_', '_'], ['X', '_', '_'], ['X', '_', '_']]

In [99]:
# Poniższy kod jest tożsamy z powyższym.
board = []
row = ["_"] * 3
for _ in range(3):
    board.append(row)

### **Niezmienność Krotek**

In [104]:
x = (1, 2, [3, 4])

try:
    x[2] += [5, 6]
except TypeError as e:
    print(e)

x

'tuple' object does not support item assignment


(1, 2, [3, 4, 5, 6])

### **Widoki Pamięci**

Wbudowana klasa `memoryview` (widok pamięci) jest typem sekwencji obejmującym współdzieloną pamięć, która pozwala obsługiwać wycinki tablic bez kopiowania bajtów. Została zainspirowana biblioteką NumPy. 

In [108]:
from array import array

x = array("b", range(6))
x

array('b', [0, 1, 2, 3, 4, 5])

In [110]:
m1 = memoryview(x)
m1.tolist()

[0, 1, 2, 3, 4, 5]

In [111]:
m2 = m1.cast("b", shape=[2, 3])
m2.tolist()

[[0, 1, 2], [3, 4, 5]]

In [113]:
m2[1, 1] = -1
x

array('b', [0, 1, 2, 3, -1, 5])

In [116]:
arr = array("b", [1, 2, 3, 4, 5, 6])
memoryview(arr).cast("b", [2, 3])[1, 1] = -100
arr

array('b', [1, 2, 3, 4, -100, 6])